In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

In [ ]:
# from DTforVec_numerical_03_2MonteCarlo import predeal, constant, tools
# from DTforVec_numerical_03_2MonteCarlo.DT_num_distance import DT_num_distance
# from DTforVec_numerical_03_2MonteCarlo.DT_num_gini import DT_num_gini
# from DTforVec_numerical_03_2MonteCarlo.DT_num_gini_monteCarlo import DT_num_gini_monteCarlo

In [ ]:
# from predeal import *
# import constant
# from tools import *
# from DT_num_distance import *
# from DT_num_gini import *
# from DT_num_gini_monteCarlo import *


In [2]:
import predeal
import constant
import tools
from DT_num_distance import *
from DT_num_gini import *
from DT_num_gini_monteCarlo import *


In [3]:
def contrastExperiment_numerical(X, y, curDepth=0, maxLeafSize=1, meanWay=None, maxDepth=1000000000):
    k = 5
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=None)
    acc = {'myTree': [], 'myTree_mc':[],'T_median':[],'T_mean':[],'standard': [], 'nearestCentroid': []}
    X_normalized = predeal.normalization(X)
    for train_index, test_index in skf.split(X_normalized, y):
        X_train_normalized, X_test_normalized = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train_normalized, y_train = shuffle(X_train_normalized, y_train)

        constant.set_value('gl_Xtrain', X_train_normalized)
        constant.set_value('gl_ytrain', y_train)
        constant.set_value('gl_distances', tools.calcDistancesMetric(X_train_normalized))

        T_gini = DT_num_gini()
        indeices = [index for index in range(len(X_train_normalized))]
        T_gini.fit(indeices)
        acc['myTree'].append(T_gini.score(X_test_normalized, y_test))

        T_gini_mc = DT_num_gini_monteCarlo()
        indeices = [index for index in range(len(X_train_normalized))]
        T_gini_mc.fit(indeices)
        acc['myTree_mc'].append(T_gini_mc.score(X_test_normalized, y_test))

        T_median=DT_num_distance(curDepth=0, maxLeafSize=1, meanWay='MEDIAN', maxDepth=1000000000)
        indeices = [index for index in range(len(X_train_normalized))]
        T_median.fit(indeices=indeices)
        acc['T_median'].append(T_median.score(X_test_normalized, y_test))

        T_mean = DT_num_distance(curDepth=0, maxLeafSize=1, meanWay='MEAN', maxDepth=1000000000)
        indeices = [index for index in range(len(X_train_normalized))]
        T_mean.fit(indeices=indeices)
        acc['T_mean'].append(T_mean.score(X_test_normalized, y_test))

        standardTree = DecisionTreeClassifier()
        standardTree.fit(X_train_normalized, y_train)
        acc['standard'].append(standardTree.score(X_test_normalized, y_test))

        ncd = NearestCentroid()
        ncd.fit(X_train_normalized, y_train)
        acc['nearestCentroid'].append(ncd.score(X_test_normalized, y_test))

    return acc

In [4]:
def printResult(acc):
    print(acc)
    # print('average-myTree: ', np.mean(acc['myTree']))
    # print('average-T_mc: ', np.mean(acc['myTree_mc']))
    # print('average-T_median: ', np.mean(acc['T_median']))
    # print('average-standard: ', np.mean(acc['standard']))
    # print('average-nearestCentroid: ', np.mean(acc['nearestCentroid']))
    print('average-myTree: %.2f , the detail is %s ' %(np.mean(acc['myTree']),acc['myTree']))
    print('average-myTree_mc: %.2f , the detail is %s ' % (np.mean(acc['myTree_mc']), acc['myTree_mc']))
    print('average-T_median: %.2f , the detail is %s ' % (np.mean(acc['T_median']), acc['T_median']))
    print('average-T_mean: %.2f , the detail is %s ' % (np.mean(acc['T_mean']), acc['T_mean']))
    print('average-standard: %.2f , the detail is %s ' % (np.mean(acc['standard']), acc['standard']))
    print('average-nearestCentroid: %.2f , the detail is %s ' % (np.mean(acc['nearestCentroid']), acc['nearestCentroid']))

In [5]:
def test_iris(curDepth=0, maxLeafSize=1, meanWay='MEDIAN', maxDepth=1000000000):
    print('iris')
    constant._init()
    from sklearn.datasets import load_iris
    iris = load_iris()
    X = iris.data
    y = iris.target
    print(X.shape)
    acc=contrastExperiment_numerical(X, y, curDepth, maxLeafSize, meanWay, maxDepth)
    printResult(acc)

In [6]:
test_iris()

iris
(150, 4)


  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

{'myTree': [0.9333333333333333, 0.8666666666666667, 0.9333333333333333, 0.9666666666666667, 0.9666666666666667], 'myTree_mc': [0.9333333333333333, 0.8666666666666667, 0.9666666666666667, 0.9666666666666667, 0.9666666666666667], 'T_median': [1.0, 0.9, 0.9, 0.9666666666666667, 0.9666666666666667], 'T_mean': [0.9666666666666667, 0.8666666666666667, 0.9666666666666667, 1.0, 0.9666666666666667], 'standard': [0.9666666666666667, 0.8666666666666667, 0.9666666666666667, 0.9666666666666667, 0.9666666666666667], 'nearestCentroid': [0.9666666666666667, 0.9, 0.9666666666666667, 0.8333333333333334, 0.9666666666666667]}
average-myTree: 0.93 , the detail is [0.9333333333333333, 0.8666666666666667, 0.9333333333333333, 0.9666666666666667, 0.9666666666666667] 
average-myTree_mc: 0.94 , the detail is [0.9333333333333333, 0.8666666666666667, 0.9666666666666667, 0.9666666666666667, 0.9666666666666667] 
average-T_median: 0.95 , the detail is [1.0, 0.9, 0.9, 0.9666666666666667, 0.9666666666666667] 
average-T